In [1]:
import pycurl
try:
    from io import BytesIO
except ImportError:
    from StringIO import StringIO as BytesIO
try:
    from urllib.parse import urlencode
except ImportError:
    from urllib import urlencode
import urllib.request
import json
import time
from socket import *

In [2]:
Base_URL = 'http://196.254.40.70/cgi/'

def sensor_do(s, url, pf, buf):
    s.setopt(s.URL, url)
    s.setopt(s.POSTFIELDS, pf)
    s.setopt(s.WRITEDATA, buf)
    s.perform()
    rcode = s.getinfo(s.RESPONSE_CODE)
    success = rcode in range(200, 207)
    print('%s %s: %d (%s)' % (url, pf, rcode, 'OK' if success else 'ERROR'))
    return success

In [3]:
sensor = pycurl.Curl()
buffer = BytesIO()
rc = sensor_do(sensor, Base_URL+'reset', urlencode({'data':'reset_system'}), buffer)
if rc:
    rc = sensor_do(sensor, Base_URL+'setting', urlencode({'rpm':'600'}), buffer)
if rc:
    rc = sensor_do(sensor, Base_URL+'setting', urlencode({'laser':'on'}), buffer)
sensor.close()

http://196.254.40.70/cgi/reset data=reset_system: 204 (OK)
http://196.254.40.70/cgi/setting rpm=600: 204 (OK)
http://196.254.40.70/cgi/setting laser=on: 204 (OK)


In [4]:
from twisted.internet.protocol import DatagramProtocol
from twisted.internet import reactor

class Echo(DatagramProtocol):
    def datagramReceived(self, data, addr):
        angle = ((data[0x91 - 42] << 8) | data[0x90  - 42])
        time = ((data[0x04DD - 42] << 24) | (data[0x04DC - 42] << 16) | (data[0x04DB - 42] << 8) | data[0x04DA - 42])
        try:
            if self.old_angle > angle:
                if 36000 - self.old_angle < angle:
                    print(f"{self.old_time} {self.old_angle}")
                else:
                    print(f"{time} {angle}")
        except AttributeError:
            pass
        
        self.old_angle = angle
        self.old_time = time

reactor.listenUDP(2368, Echo())
reactor.run()

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'service_identity''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


300599962 27
300699495 35838
300800355 133
300899887 35955
300999420 35788
301100280 114
301199813 35960
301299346 35791
301400205 95
301499738 35922
301600598 234
301700131 85
301799664 35940
301899196 35786
302000056 101
302099589 35932
302200449 230
302299982 50
302399515 35877
302500374 189
302599907 27
302699440 35855
302800300 149
302899833 35957
302999365 35772
303100225 75
303199758 35912
303300618 233
303400151 85
303499684 35927
303600543 239
303700076 75
303799609 35912
303900469 218
304000002 52
304099534 35900
304200394 232
304299927 73
304399460 35898
304500320 191
304599853 35999
304699385 35820
304800245 130
304899778 35969
304999311 35804
305100171 109
305199703 35926
305300563 225
305400096 51
305499629 35879
305600489 188
305700022 19
305799554 35849
305900414 154
305999947 35984
306099480 35812
306200340 130
306299872 35982
306399405 35834
306500265 157
306599798 35992
306699331 35819
306800190 127
306899723 35957
306999256 35790
307100116 107
307199649 35943
307299

In [5]:
sensor = pycurl.Curl()
buffer = BytesIO()
rc = sensor_do(sensor, Base_URL+'reset', urlencode({'data':'reset_system'}), buffer)
sensor.close()

http://196.254.40.70/cgi/reset data=reset_system: 204 (OK)
